In [ ]:
pip install scgraph>=2.4.0
pip install folium==0.16.0

In [2]:
from scgraph.geographs.marnet import marnet_geograph

# Get the shortest path between shanghai and savannah
marnet_output = marnet_geograph.get_shortest_path(
    origin_node={"latitude": 31.23,"longitude": 121.47}, 
    destination_node={"latitude": 32.08,"longitude": -81.09}
)
print('Length: ',marnet_output['length'])
print('Path: ', marnet_output['coordinate_path'])

Length:  19072.8293
Path:  [[31.23, 121.47], [30.7324, 121.8274], [30.8345, 122.6776], [31.3, 122.9], [33.5458, 126.5397], [34.2, 127.6], [35.0, 129.2], [38.2106, 134.0843], [40.0, 137.2662], [41.2, 139.4], [41.9, 141.25], [41.9, 143.3], [43.2, 146.0], [46.0975, 153.3524], [46.1961, 153.6877], [47.4449, 157.9333], [48.1099, 160.1942], [48.3248, 160.925], [48.4755, 161.4373], [50.0, 169.0188], [50.1542, 169.7857], [50.2383, 170.2039], [50.4839, 172.3546], [50.6878, 174.1401], [51.3, 179.5], [51.6, -176.7], [51.2951, -171.5552], [51.0966, -168.2056], [51.0799, -168.081], [50.7515, -165.623], [50.3734, -162.7942], [50.0, -160.0], [49.1714, -154.5354], [48.9508, -153.4555], [48.5698, -151.5903], [48.2452, -150.0011], [48.0972, -149.2766], [47.6332, -147.484], [46.7971, -144.2543], [46.6354, -143.7413], [45.4566, -140.0027], [45.2935, -139.4855], [44.1117, -136.3287], [43.6057, -134.9769], [43.4858, -134.7002], [42.4942, -132.4113], [42.4421, -132.2909], [40.8893, -130.8343], [40.0, -130.0]

In [3]:
from scgraph.geographs.oak_ridge_maritime import oak_ridge_maritime_geograph

# Get the shortest path between shanghai and savannah
oak_ridge_output = oak_ridge_maritime_geograph.get_shortest_path(
    origin_node={"latitude": 31.23,"longitude": 121.47}, 
    destination_node={"latitude": 32.08,"longitude": -81.09}
)
print('Length: ',oak_ridge_output['length'])
print('Path: ', oak_ridge_output['coordinate_path'])

Length:  19168.772
Path:  [[31.23, 121.47], [31.3, 122.9], [34.2, 127.6], [35.0, 129.2], [38.2106, 134.0843], [41.2, 139.4], [41.9, 141.25], [41.9, 143.3], [43.2, 146.0], [46.0975, 153.3524], [48.4755, 161.4373], [50.2383, 170.2039], [51.3, 179.5], [51.4362, -178.9746], [51.6, -176.7], [51.0966, -168.2056], [50.0, -160.0], [49.1714, -154.5354], [48.0972, -149.2766], [46.7971, -144.2543], [45.2935, -139.4855], [43.6057, -134.9769], [41.7617, -130.7204], [39.7846, -126.7021], [37.7, -122.9], [37.4364, -123.4028], [37.2792, -123.317], [36.9315, -123.1281], [36.8027, -123.0581], [36.6328, -122.9663], [35.4383, -122.0464], [35.3616, -122.0005], [35.139, -121.8686], [34.5768, -121.532], [34.4361, -120.8841], [34.3051, -120.5192], [34.2522, -120.2844], [34.1977, -120.0414], [34.1628, -119.8891], [34.1473, -119.8221], [34.1359, -119.7688], [34.0645, -119.4593], [34.0319, -119.2946], [33.6364, -118.4648], [33.6384, -118.295], [33.6279, -118.1788], [33.5632, -118.1609], [33.3191, -118.0937], [33

In [4]:
# A simple example of how to use folium to visualize the shortest path between two points
import folium
from scgraph.geographs.marnet import marnet_geograph

# Get the shortest path between shanghai and houston
folium_output = marnet_geograph.get_shortest_path(
    origin_node={"latitude": 31.23,"longitude": 121.47}, 
    destination_node={"latitude": 29.76,"longitude": -95.37}
)

# Folium does not wrap coordinates around the globe, so we need to adjust the path accordingly
# Essentially we create 5 differnt paths that offset the original path by 0, 360, -360, 720, -720 degrees
# This allows the illusion that the path wraps around the globe with folium
def adjustArcPath(path):
    for index in range(1, len(path)):
        x = path[index][1]
        prevX = path[index - 1][1]
        path[index][1] = x - (round((x - prevX)/360,0) * 360)
    return path

def modifyArcPathLong(points, amount):
    return [[i[0], i[1]+amount] for i in points]

def getCleanArcPath(path):
    path = adjustArcPath(path)
    return [
        path,
        modifyArcPathLong(path, 360),
        modifyArcPathLong(path, -360),
        modifyArcPathLong(path, 720),
        modifyArcPathLong(path, -720)
    ]

# Create a folium map
map = folium.Map([40, 180], zoom_start=2)
# Populate it with the path
folium.PolyLine(
    getCleanArcPath(folium_output['coordinate_path']),
    color='green',
    weight=5,
    opacity=0.5,
    popup='Length (KM): ' + str(folium_output['length'])
).add_to(map)
map